In [1]:
import warnings
warnings.filterwarnings('ignore')

!pip install pydicom
import pandas as pd
import numpy as np
import random
import csv
import os

import pydicom,cv2,pylab
import seaborn as sns
import pydicom as dcm

from skimage import io
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
import keras.backend as K
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator



from matplotlib import pyplot as plt
import matplotlib.patches as patches


     |████████████████████████████████| 35.3MB 119kB/s 


Using TensorFlow backend.


In [2]:
#Mounting drive
from google.colab import drive
drive.mount('./content/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./content/


In [0]:
#Defining project path
project_path = "./content/My Drive/"

In [4]:
#Unzippiong data file
from zipfile import ZipFile
with ZipFile(project_path+'rsna-pneumonia-detection-challenge.zip', 'r') as z:
  z.extractall()

OSError: ignored

In [0]:
import keras
import tensorflow as tf
from keras import regularizers
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import Model, Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.applications.densenet import DenseNet121
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix,roc_auc_score



In [0]:
os.getcwd()

'/content'

In [0]:
# load and shuffle filenames
folder_test = '/content/stage_2_test_images'
test_filenames = os.listdir(folder_test)
print('n test samples:', len(test_filenames))

n test samples: 3000


In [0]:
random.shuffle(test_filenames)
samples = 100
test_samples = test_filenames[:samples]

MODEL 1 : CHEXNET CLASSIFICATION MODEL 

In [0]:
image_size = 256
# Constant parameters 
input_shape = (image_size, image_size, 3)
num_classes = 1

#Initialize the model

inputs = Input(shape=input_shape)

chexNet = DenseNet121(
          include_top=True
        , input_tensor=inputs
        , weights="./content/My Drive/ChexNet_weight.h5"
        , classes=14
    )
 
chexNet = Model(
          inputs=inputs
        , outputs=chexNet.layers[-2].output
        , name="ChexNet"
    )
 # Weights downloaded from https://github.com/erniechiew/kaggle_rsna_pneumonia_dancingbears/tree/master/classification 

class_model = Sequential()

class_model.add(chexNet)

class_model.add(Dropout(0.6, name="drop_0"))
#model.add(Dense(512, activation = 'relu', name = "dense_0", kernel_regularizer = regularizers.l2(0.01)))
class_model.add(Dense(512, activation = 'relu', name = 'dense_1', kernel_regularizer = regularizers.l2(0.01)))
class_model.add(Dense(256, activation = None, name = 'dense_2', kernel_regularizer = regularizers.l2(0.01)))
class_model.add(LeakyReLU(alpha=0.3))
class_model.add(Dropout(0.6, name="drop_2"))
class_model.add(Dense(1, activation="sigmoid", name="out"))



class_model.summary()















Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ChexNet (Model)              (None, 1024)              7037504   
_________________________________________________________________
drop_0 (Dropout)             (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
drop_2 (Dropout)             (None, 256)               0  

MODEL 2 : CNN SEGMENTATION FOR BOUNDING BOX PREDICTION MODEL

In [0]:
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=4):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
            # output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model_bb = keras.Model(inputs=inputs, outputs=outputs)
    return model_bb

# create network and compiler
model_bb = create_network(input_size = 256, channels=32)
model_bb.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 288         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_2 (LeakyReLU)       (None, 256, 256, 32) 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

LOADING CLASSIFICATION MODEL WEIGTHS

In [0]:
class_model.load_weights('/content/content/My Drive/classifier_768_SGD.h5')

In [0]:
class_model.load_weights('/content/content/My Drive/FINAL_CLASSIFIER_WEIGHTS.h5')

LOADING CNN SEGMENTATION BOUNDING BOX MODEL

In [0]:
model_bb.load_weights('/content/content/My Drive/classifier_bb_256.h5')


In [0]:
model_bb.load_weights('/content/content/My Drive/FINAL WEIGHTS MODEL 2.h5')


- CLASSIFICATION DETECTION MODEL FUNCTION 


In [0]:
from skimage.measure import label,regionprops

#Initializing Dictionary
submission_dict = {}
# loop through testset
def classification_detection(filenames,classification_threshold, detection_threshold):
    negative = 0
    positive = 0
    for filename in filenames:
        img = pydicom.dcmread(os.path.join(folder_test, filename)).pixel_array # reading .dicom images
        # resize image
        img = resize(img, (image_size,image_size), mode='reflect')
        #Normalize
        img = img/255
        #adding trailing dimensions
        img_rgb = np.broadcast_to(img[...,np.newaxis],(1,image_size,image_size,3)) # Converting to 'rgb' for using with ChexNet
        y = class_model.predict(img_rgb) #Prediction using ChexNet Prediction Model
        if y < classification_threshold: # fixing threshold for classification
          predictionString = ''
          negative = negative + 1
          filename = filename.split('.')[0]
          submission_dict[filename] = ''
        else:
          positive = positive + 1
          #print(r'positive : {}'.format(positive))
          img = np.broadcast_to(img[...,np.newaxis],(1,image_size,image_size,1)) 
          img = img*255
          #print(img.shape)
          #Predicting bounding box
          pred = model_bb.predict(img)
          patientId = filename
          for pred in pred:
            # resize predicted mask
            pred = resize(pred, (1024, 1024), mode='reflect') #Resizing to 1024 x 1024
            # threshold predicted mask
            comp = pred[:, :, 0] > detection_threshold #fixing threshold for detection
            # apply connected components
            comp = measure.label(comp)
            # apply bounding boxes
            predictionString = ''
            for region in measure.regionprops(comp):
              # retrieve x, y, height and width
              y, x, y2, x2 = region.bbox
              height = y2 - y
              width = x2 - x
              # proxy for confidence score
              conf = np.mean(pred[y:y+height, x:x+width])
              # add to predictionString
              predictionString += str(conf)  + ' ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) +  ' '
            #add filename and predictionString to dictionary
            patientId = patientId.split('.')[0]
            #print(patientId)
            submission_dict[patientId] = predictionString
            # stop if we've got them all
          if len(submission_dict) == len(test_filenames):
            return submission_dict, positive

In [0]:
classification_detection(test_filenames,0.4,0.4)

In [0]:
submission_dict

{'2ae28df1-2084-4000-9327-177953f4b18c': '',
 '2021bb8f-9fad-4220-b7a4-73894f5dbc19': '0.5948978 256 192 512 703 ',
 '229bb456-7c14-402a-a61f-fc2e5aac9729': '',
 '1532c43a-028c-4d77-9cb0-f9d103988e7b': '',
 '25e615d9-5bb0-46da-8625-0aa2ae1c64e9': '',
 '037f16ff-46b6-45bf-b737-74072c46fab7': '',
 '23f35b71-b93b-4f74-a9ca-579f188cec2b': '',
 '1a0309e4-762b-4b64-9e2c-0f16dc4d5381': '',
 '21ab7839-a62c-4829-bde0-378d0ca4cc21': '',
 '0e064c00-ba9b-49ba-af5b-cf52d819604c': '',
 '2dc40698-93d3-4310-a9f8-d8ee88cac98f': '',
 '296939fd-29fe-495d-a7dc-f7127240a9c8': '',
 '28f51877-4cf3-4476-af15-256738626d14': '',
 '1dd81458-239d-4177-93a1-536b1f127d19': '',
 '1fe96923-d9d0-4be7-8aa7-48b3ab14628f': '0.8188126 191 640 193 192 0.98183787 576 640 192 129 ',
 '0f165be0-0173-4a7c-8411-9b9b29e4f021': '',
 '1189f742-0450-455c-8311-192da30f23b8': '0.4799742 640 385 192 383 0.7815207 256 448 256 320 ',
 '059cdbab-5cdc-4b42-bf37-0423ae3eba0a': '',
 '0f162616-cc4a-46fe-a7a9-3637fa8889f5': '',
 '30956603-11a

In [0]:
len(submission_dict)

3000

In [0]:
# save dictionary as csv file
sub = pd.DataFrame.from_dict(submission_dict, orient = 'index')
sub.index.names = ['PatientId']
sub.columns = ['PredictionString']
sub.reset_index()



,PatientId,PredictionString
0,2ae28df1-2084-4000-9327-177953f4b18c,
1,2021bb8f-9fad-4220-b7a4-73894f5dbc19,0.5948978 256 192 512 703
2,229bb456-7c14-402a-a61f-fc2e5aac9729,
3,1532c43a-028c-4d77-9cb0-f9d103988e7b,
4,25e615d9-5bb0-46da-8625-0aa2ae1c64e9,
...,...,...
2995,0467d282-4548-4430-bc98-1c3dbfc96e8c,0.5927751 64 383 896 641
2996,1cd3a288-dc8e-4591-9667-d4f1f5b48c79,
2997,0cf8860b-ee52-4b64-b8a6-5f753c64412b,0.89730537 128 256 256 512 0.89998096 576 384 ...
2998,c1db263b-5d0e-47f7-9093-b7ad5b2a4d2a,0.611997 64 320 896 704


In [0]:
sub[sub.iloc[:,0] == '']

,PredictionString
PatientId,
2ae28df1-2084-4000-9327-177953f4b18c,
229bb456-7c14-402a-a61f-fc2e5aac9729,
1532c43a-028c-4d77-9cb0-f9d103988e7b,
25e615d9-5bb0-46da-8625-0aa2ae1c64e9,
037f16ff-46b6-45bf-b737-74072c46fab7,
...,...
2e26ac7e-9271-4819-a86d-6139d6ceedb7,
2c61626f-bbbc-4081-950a-e11463990eca,
203b1b72-8b45-469a-b023-3a460f360a34,
